# Problem regresji - Boston Housing Price Prediction
### Spis treści:
[1. Ładowanie danych i wstępna eksploracja](#sekcja-1)<br>
[2. Przygotowanie zbioru treningowego i testowego](#sekcja-2)<br>
[3. Standaryzacja danych](#sekcja-3)<br>
[4. Budowa modelu](#sekcja-4)<br>
[5. Ocena modelu](#sekcja-5)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
import plotly.express as px

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format='{:.6f}'.format

In [ ]:
tf.__version__

'2.13.0'

<a name="sekcja-1"></a>
### 1. Ładowanie danych i wstępna eksploracja

In [ ]:
raw = pd.read_csv("/content/drive/MyDrive/Uczenie Maszynowe/Notatniki/housing.csv")
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [ ]:
data = raw.copy()
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY


In [ ]:
data.tail()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20635,-121.090000,39.480000,25.000000,1665.000000,374.000000,845.000000,330.000000,1.560300,78100.000000,INLAND
20636,-121.210000,39.490000,18.000000,697.000000,150.000000,356.000000,114.000000,2.556800,77100.000000,INLAND
20637,-121.220000,39.430000,17.000000,2254.000000,485.000000,1007.000000,433.000000,1.700000,92300.000000,INLAND
20638,-121.320000,39.430000,18.000000,1860.000000,409.000000,741.000000,349.000000,1.867200,84700.000000,INLAND
20639,-121.240000,39.370000,16.000000,2785.000000,616.000000,1387.000000,530.000000,2.388600,89400.000000,INLAND


In [ ]:
raw.isnull().sum() / len(raw)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.010029
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [ ]:
data.dropna(inplace=True)
data.isnull().sum() / len(data)

longitude            0.000000
latitude             0.000000
housing_median_age   0.000000
total_rooms          0.000000
total_bedrooms       0.000000
population           0.000000
households           0.000000
median_income        0.000000
median_house_value   0.000000
ocean_proximity      0.000000
dtype: float64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,20433.000000,-119.570689,2.003578,-124.350000,-121.800000,-118.490000,-118.010000,-114.310000
latitude,20433.000000,35.633221,2.136348,32.540000,33.930000,34.260000,37.720000,41.950000
housing_median_age,20433.000000,28.633094,12.591805,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,20433.000000,2636.504233,2185.269567,2.000000,1450.000000,2127.000000,3143.000000,39320.000000
total_bedrooms,20433.000000,537.870553,421.385070,1.000000,296.000000,435.000000,647.000000,6445.000000
population,20433.000000,1424.946949,1133.208490,3.000000,787.000000,1166.000000,1722.000000,35682.000000
households,20433.000000,499.433465,382.299226,1.000000,280.000000,409.000000,604.000000,6082.000000
median_income,20433.000000,3.871162,1.899291,0.499900,2.563700,3.536500,4.744000,15.000100
median_house_value,20433.000000,206864.413155,115435.667099,14999.000000,119500.000000,179700.000000,264700.000000,500001.000000


In [ ]:
data.describe(include=["object"])

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [ ]:
data["ocean_proximity"].value_counts()

<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: ocean_proximity, dtype: int64

In [ ]:
px.histogram(data, x="median_house_value")

In [ ]:
data["median_house_value"].value_counts()

500001.000000    958
137500.000000    119
162500.000000    116
112500.000000    103
187500.000000     92
                ... 
359200.000000      1
51200.000000       1
39800.000000       1
377600.000000      1
47000.000000       1
Name: median_house_value, Length: 3833, dtype: int64

In [ ]:
data = data[data["median_house_value"] != 500001]

In [ ]:
px.histogram(data, x="median_house_value")

In [ ]:
data_dummies = pd.get_dummies(data=data, drop_first=True)
data_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,0,0,1,0
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,0,0,1,0
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,0,0,1,0
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,0,0,1,0
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,0,0,1,0


<a name="sekcja-2"></a>
### 2. Przygotowanie zbioru treningowego i testowego

In [ ]:
train_dataset = data_dummies.sample(frac=0.8, random_state=0)
test_dataset = data_dummies.drop(train_dataset.index)

print(f"Zbiór treningowy: {train_dataset.shape}")
print(f"Zbiór testowy: {test_dataset.shape}")

Zbiór treningowy: (15580, 13)
Zbiór testowy: (3895, 13)


In [ ]:
train_label = train_dataset.pop("median_house_value").to_numpy()
test_label = test_dataset.pop("median_house_value").to_numpy()

<a name="sekcja-3"></a>
### 3. Standaryzacja danych

In [ ]:
from sklearn.preprocessing import StandardScaler

skaler = StandardScaler()

normed_train_dataset = skaler.fit_transform(train_dataset)
normed_test_dataset = skaler.fit_transform(test_dataset)

<a name="sekcja-4"></a>
### 4. Budowa modelu

In [ ]:
def nn_model():
    model = Sequential()
    model.add(Dense(1024, kernel_regularizer="l2", activation="relu", input_shape=[len(train_dataset.keys())]))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer="adam",
                loss="mse",
                metrics=["mae", "mse"])

    return model

In [ ]:
model = nn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              13312     
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor="val_loss", mode="min", patience=4, verbose=1)

history = model.fit(normed_train_dataset, train_label, epochs=1000, validation_split=0.2, batch_size=32, verbose=1, callbacks=[es])

Epoch 1/1000
390/390 [==============================] - 9s 18ms/step - loss: 17126267904.0000 - mae: 95444.6719 - mse: 17126267904.0000 - val_loss: 4574177792.0000 - val_mae: 48820.9688 - val_mse: 4574177792.0000
Epoch 2/1000
390/390 [==============================] - 7s 19ms/step - loss: 4423407616.0000 - mae: 47839.1367 - mse: 4423407616.0000 - val_loss: 3792577792.0000 - val_mae: 44682.6406 - val_mse: 3792577792.0000
Epoch 3/1000
390/390 [==============================] - 7s 17ms/step - loss: 3997463040.0000 - mae: 45505.5352 - mse: 3997463040.0000 - val_loss: 3608647168.0000 - val_mae: 43764.7617 - val_mse: 3608647168.0000
Epoch 4/1000
390/390 [==============================] - 7s 17ms/step - loss: 3815079168.0000 - mae: 44475.3086 - mse: 3815079168.0000 - val_loss: 3614172160.0000 - val_mae: 44225.2734 - val_mse: 3614172160.0000
Epoch 5/1000
390/390 [==============================] - 7s 18ms/step - loss: 3780323328.0000 - mae: 44100.9922 - mse: 3780323328.0000 - val_loss: 34603714

In [ ]:
df_hist = pd.DataFrame(history.history)
df_hist["epoch"] = history.epoch
df_hist["rmse"] = np.sqrt(df_hist["mse"])
df_hist["val_rmse"] = np.sqrt(df_hist["val_mse"])
df_hist.head()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch,rmse,val_rmse
0,17126267904.000000,95444.671875,17126267904.000000,4574177792.000000,48820.968750,4574177792.000000,0,130867.367606,67632.668083
1,4423407616.000000,47839.136719,4423407616.000000,3792577792.000000,44682.640625,3792577792.000000,1,66508.703310,61583.908548
2,3997463040.000000,45505.535156,3997463040.000000,3608647168.000000,43764.761719,3608647168.000000,2,63225.493592,60072.016514
3,3815079168.000000,44475.308594,3815079168.000000,3614172160.000000,44225.273438,3614172160.000000,3,61766.327137,60117.985329
4,3780323328.000000,44100.992188,3780323328.000000,3460371456.000000,42805.605469,3460371456.000000,4,61484.334005,58824.922065


<a name="sekcja-5"></a>
### 5. Ocena modelu

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig1 = go.Scatter(x=df_hist["epoch"], y=df_hist["rmse"], name="rmse", mode="markers+lines")
fig2 = go.Scatter(x=df_hist["epoch"], y=df_hist["val_rmse"], name="val_rmse", mode="markers+lines")

fig.add_trace(fig1)
fig.add_trace(fig2)
fig.update_layout(width=1000, height=500, title="RMSE vs. val_RMSE", xaxis_title="Epochs", yaxis_title="RMSE")
fig.show()

In [ ]:
model.evaluate(normed_test_dataset, test_label)

122/122 [==============================] - 1s 5ms/step - loss: 2636787456.0000 - mae: 36022.7148 - mse: 2636787456.0000


[2636787456.0, 36022.71484375, 2636787456.0]

In [ ]:
for name, value in zip(model.metrics_names, model.evaluate(normed_test_dataset, test_label)):
    print(f"Metric: {name} = {value}")

122/122 [==============================] - 1s 4ms/step - loss: 2636787456.0000 - mae: 36022.7148 - mse: 2636787456.0000
Metric: loss = 2636787456.0
Metric: mae = 36022.71484375
Metric: mse = 2636787456.0


In [ ]:
predictions = model.predict(normed_test_dataset)

122/122 [==============================] - 1s 4ms/step


In [ ]:
df = pd.DataFrame(test_label, columns=["median_house_value"])
df["predictions"] = predictions
df.head()

,median_house_value,predictions
0,281500.000000,249149.343750
1,191300.000000,210429.671875
2,162900.000000,192335.687500
3,93800.000000,172321.343750
4,155400.000000,187460.031250


In [ ]:
fig = px.scatter(df, "median_house_value", "predictions")
fig.add_trace(go.Scatter(x=[0,500000], y=[0,500000], mode="lines"))

In [ ]:
df = pd.DataFrame(test_label, columns=["median_house_value"])
df["predictions"] = predictions
df["error"] = df["median_house_value"] - df["predictions"]
df.head()

,median_house_value,predictions,error
0,281500.000000,249149.343750,32350.656250
1,191300.000000,210429.671875,-19129.671875
2,162900.000000,192335.687500,-29435.687500
3,93800.000000,172321.343750,-78521.343750
4,155400.000000,187460.031250,-32060.031250


In [ ]:
px.histogram(df, x="error", marginal="rug", width=1000)